# Clustering analysis

There are two datasets that have similar information for France and South Korea, respectively. So, the idea is to merge those datasets and see if we can find clusters. 

You can find the links to the data here: 
- France: https://www.kaggle.com/lperez/coronavirus-france-dataset
- South Korea: https://www.kaggle.com/kimjihoo/coronavirusdataset

In [ ]:
import pandas as pd
import numpy as np
import random
from sklearn import preprocessing
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from kneed import KneeLocator
import geopandas
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

## Load the data and check missing values
Let's load the data for both datasets and see how many NAs values it has. NAs values corresponds to missing or empty information. Decision should be made for each particular column to prepare the data for the analysis. 

In [ ]:
def load_data(data_path):
    df = pd.read_csv(data_path + '/patient.csv')
    df['released_date'] = pd.to_datetime(df['released_date'])
    df['confirmed_date'] = pd.to_datetime(df['confirmed_date'])
    df['month'] = df['confirmed_date'].dt.month
    df['day'] = df['confirmed_date'].dt.day
    return df

In [ ]:
df_france = load_data('coronavirusdataset_france')
df_france.isnull().sum()

In [ ]:
df_south_korea = load_data('coronavirusdataset_south_korea')
df_south_korea.isnull().sum()

## Resolve missing values

- With the objective of keeping the necesary data, some of the columns are removed such as department, comments, health consiedered not important for the analysis. 
- From the birth date, we can create the age variable, substracting it to the actual date. The missing information will be filled with random numbers draw from a distribution. The information related to the age distribution of the population can be found:     
 *France:* https://www.indexmundi.com/france/demographics_profile.html  
 *South Korea:* https://www.indexmundi.com/south_korea/demographics_profile.html
 
So, the 'simulate_age' function is create in order to simulate the population's age based on the available data. 
 

In [ ]:
df_france.drop(['departement','region','comments', 'id', 'infected_by','health','city','source'],axis=1,inplace=True)
df_south_korea.drop(['region','disease','patient_id','infected_by'], axis=1, inplace=True)

In [ ]:
def simulate_age(ranges, percents, total_pop):
    simulated_pop = np.array(0)
    for (low, high), percent in zip(ranges, percents):
        simulated_pop = np.append(simulated_pop, 
                  np.random.randint(low=low, high=high, size=int(total_pop*percent/100)))
    return simulated_pop

In [ ]:
#France
france_population = 67364357

'''
0-14 years: 18.48% 
15-24 years: 11.8% 
25-54 years: 37.48% 
55-64 years: 12.42%
65 years and over: 19.82%
'''
ranges = [(0,14),(15,24),(25,54),(55,64),(65,90)]
percents = [18.48,11.8,37.48,12.42,19.82]
france_simulated_pop = simulate_age(ranges, percents, france_population)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,5))
ax1.hist(france_simulated_pop,bins=20, color='mediumaquamarine', edgecolor='k', alpha=0.5)
ax1.set_title('France - Simulated age distribution')

#South Korea
south_korea_population = 51418097

'''
0-14 years: 13.03% 
15-24 years: 12.19%
25-54 years: 45.13%
55-64 years: 15.09% 
65 years and over: 14.55% 
'''
percents = [13.03,12.19,45.13,15.09,14.55]
south_korea_simulated_pop = simulate_age(ranges, percents, south_korea_population)

ax2.hist(south_korea_simulated_pop,bins=20, color='mediumaquamarine', edgecolor='k', alpha=0.5)
ax2.set_title('South Korea - Simulated age distribution')
plt.show()

Now, let's create the age column, and fill the missing values with a random value choosen from the distributions that we just simulated.

In [ ]:
import math
actual_year = pd.to_datetime('today').year
def calculate_age(x):
    if math.isnan(x):
        return x
    else:
        return int(actual_year - x)

#France
df_france['age'] = df_france['birth_year'].apply(calculate_age)
df_france.fillna({'age':int(random.choice(france_simulated_pop))}, inplace=True)
df_france.drop(['birth_year'], axis=1, inplace=True)

#South Korea
df_south_korea['age'] = df_south_korea['birth_year'].apply(calculate_age)
df_south_korea.fillna({'age':int(random.choice(south_korea_simulated_pop))}, inplace=True)
df_south_korea.drop(['birth_year'], axis=1, inplace=True)

For sex missing values, we can draw a random number with a value of probability based on the sex ratio for each population.

In [ ]:

'''
Considering m as men and w as women. 
m/w=ratio -> m=ration*w
m+w=total_pop
-> ratio*w +w=total_pop -> (ratio+1)*w=total_pop -> w=total_pop/(ratio+1)
we should divide w by the total in order to get the probability of being women 
'''
def calculate_values(ratio, total_pop):
    w = (france_population/(1+ratio))/total_pop
    m = 1 - w
    return (w,m)


# France
# total population: 0.96 male(s)/female (2018 est.)
w,m = calculate_values(0.96, france_population)
#choice among 0 (woman) and 1 (man) with the calculated probabilities
df_france['sex'] = df_france['sex'].str.lower()
df_france["sex"].replace({"male\xa0?": "male"}, inplace=True)
df_france.fillna({'sex': np.random.choice(['female','male'],p=[w,m])}, inplace=True)

# South Korea
# total population: 1 male(s)/female (2018 est.)
w,m = calculate_values(1, south_korea_population)
df_south_korea['sex'] = df_south_korea['sex'].str.lower()
df_south_korea["sex"].replace({"male\xa0?": "male"}, inplace=True)
df_south_korea.fillna({'sex': np.random.choice(['female','male'],p=[w,m])}, inplace=True)

Since the status column for France's dataset and the state column for South Korea's dataset have the same meaning, we can rename the column for one of the datasets, and update the values to be the same categories.

In [ ]:
df_france.rename({'status':'state'}, axis=1, inplace=True)
df_france['state'] = df_france['state'].apply(lambda x: 'isolated' if (x=='hospital' or x=='home isolation') else x)

- The values for the country variable that are empty will be filled with France or South Korea, respectevily.
- A new category 'Unkown' will be created for infection_reason, group, status variables
- A new category for infection_order is added with code 0 
- The empty values for contact number will be filled with 0 

In [ ]:
df_france.fillna({'country':'France','infection_reason':'Unkown','group':'Unkown', 
                  'state':'Unknown','infection_order':0, 'contact_number':0} ,
                 inplace=True)

df_south_korea.fillna({'infection_reason':'Unkown','group':'Unkown', 
                       'infection_order':0, 'contact_number':0, 
                       'state':'Unknown'} ,
                 inplace=True)

Let's check now which are the missing values that still  need to be resolved.

In [ ]:
df_france.isnull().sum()

In [ ]:
df_south_korea.isnull().sum()

Nice!, we don't have too much left. 
Now we need to resolve released_date and deceased_date empty values. 
- If released_date is empty, it means that the person still has the virus. 
- If deceased_date is empty, it means that the person did not died. 

So, we can calculate the infection duration in days and remove the other 3 variables.  
And transform deceased_date to a binary column, indicated if the person died or not.

In [ ]:
df_france['released_date'] = df_france[['released_date','deceased_date']].fillna(df_france['deceased_date'])
df_france['released_date'] = df_france[['released_date']].fillna(pd.to_datetime('today'))
df_france['infection_duration'] = pd.to_datetime(df_france['released_date']).sub(df_france['confirmed_date'], axis=0)
df_france = df_france[df_france['infection_duration'].dt.days>=0]
df_france['infection_duration'] = df_france['infection_duration'].dt.days
df_france.drop(['released_date','confirmed_date','deceased_date'], axis=1, inplace=True)

df_south_korea['released_date'] = df_south_korea[['released_date','deceased_date']].fillna(df_south_korea['deceased_date'])
df_south_korea['released_date'] = df_south_korea[['released_date']].fillna(pd.to_datetime('today'))
df_south_korea['infection_duration'] = pd.to_datetime(df_south_korea['released_date']).sub(df_south_korea['confirmed_date'], axis=0)
df_south_korea = df_south_korea[df_south_korea['infection_duration'].dt.days>=0]
df_south_korea['infection_duration'] = df_south_korea['infection_duration'].dt.days
df_south_korea.drop(['released_date','confirmed_date','deceased_date'], axis=1, inplace=True)

In [ ]:
df_france.columns

In [ ]:
df_south_korea.columns

## Data Fusion
Finally, we are ready to put the two datasets together and start our analysis.

In [ ]:
df = df_france.append(df_south_korea, sort=False)

In [ ]:
df.isnull().sum()

## Transform to dummies the categorical variables

In [ ]:
df = pd.concat([df, pd.get_dummies(df['sex'])], axis=1)
df = pd.concat([df, pd.get_dummies(df['country'])], axis=1)
df = pd.concat([df, pd.get_dummies(df['state'], drop_first=True)], axis=1)
df = pd.concat([df, pd.get_dummies(df['infection_reason'], drop_first=True)], axis=1)
df = pd.concat([df, pd.get_dummies(df['group'], drop_first=True)], axis=1)

## Dimension reduction

Since we have too many variables, it is difficult to find the pattern among the clusters. 
So, first we can reduce the number of categorical variables by groupping similar categories. 
After, we can apply a dimension reduction technique to reduce the input variables and make the model easier to interpret. 

In [ ]:
df = df_france.append(df_south_korea, sort=False)

Transform infection_reason: let's list the possible values for this variable, and group them. The, transform to dummy variables. Drop one since it is implicit from the others. 

In [ ]:
df.infection_reason.unique()

In [ ]:
def transform_reason(value):
    if ('religious' in value or 'parishioner' in value):
        return 'religious'
    elif ('visit' in value or 'residence' in value):
        return 'visit'
    elif ('contact' in value):
        return 'contact'
    elif ('medical' in value or 'health professional' in value):
        return 'medical'
    elif ('militar' in value):
        return 'militar'
    elif ('italian' in value):
        return 'italian'
    elif ('pilgrimage' in value):
        return 'pilgrimage'
    else:
        return 'unknown'

df['infection_reason'] = df['infection_reason'].str.lower()
df['infection_reason'] = df['infection_reason'].apply(transform_reason)  
df = pd.concat([df, pd.get_dummies(df['infection_reason'], prefix='infection_reason', prefix_sep='_')], axis=1)
df.drop(['infection_reason_unknown'], axis=1, inplace=True)

Since the 'group' variable provides similar information to infection_reson, it will be removed. 

In [ ]:
df.drop(['group'], axis=1, inplace=True)

Let's transform the other categorical variables to dummies: country, state and sex.

In [ ]:
df = pd.concat([df, pd.get_dummies(df['country'])], axis=1)
df = pd.concat([df, pd.get_dummies(df['state'], prefix='state', prefix_sep='_')], axis=1)
df = pd.concat([df, pd.get_dummies(df['sex'])], axis=1)

In [ ]:
features = df.drop(['country','state','sex','infection_reason'], axis=1)
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
x = StandardScaler().fit_transform(features.values)
pca = PCA(random_state=0)
pca.fit(x)

In [ ]:
#determine number of components with threshold=0.8
n_components=np.where(np.cumsum(pca.explained_variance_ratio_)>0.8)[0][0]+1
#explained variance
v = round(np.cumsum(pca.explained_variance_ratio_)[n_components-1]*100,1)
print(f'It is needed {n_components} components to explain {v}% variance of the data')

In [ ]:
pca = PCA(n_components=n_components)
pcs = pca.fit(x)
components_name = list(range(0, n_components))
components_name = list(map(lambda x: 'PC' + str(x), components_name))
pd.DataFrame(data=pcs.components_, columns = features.columns, index=components_name)

In [ ]:
components_range = np.arange(1, n_components+1, 1)
components_names = list(map(lambda x: 'PC' + str(x), components_range))
plt.matshow(pcs.components_,cmap='viridis')
plt.yticks(range(0,n_components), components_names,fontsize=10)
plt.colorbar()
plt.xticks(range(0,len(features.columns)),features.columns,rotation=90,ha='left')
plt.show()

Higer values for the variables means more influence in the principal component. Lower values, means negative influence in the principal components. 
From the matrix, one possible interpretation for the principal componentes is:
- PC1: female who is not isolated and is not from from Korea
- PC2: first months 
- PC3: state released
- PC4: state deceased
- PC5: infection reason religiuos
- PC6: low infection duration 
- PC7: infection reason italian
- PC8: infection reason militar
- PC9: infection reason medical
- PC10: infection reason piligrimage
- PC11: high infection order, not from China 
- PC12: high contact number, not from Mongolia

## Kmeans Clustering

In [ ]:
def elbow_test(df, n_init, max_clusters, max_iter):
    distortions = []
    for i in range(1, max_clusters):
        km = KMeans(
            n_clusters=i, init='random',
            n_init=n_init, max_iter=max_iter,
            tol=1e-04, random_state=0
        )
        km.fit(df)
        distortions.append(km.inertia_)

    plt.plot(range(1, max_clusters), distortions, marker='o')
    plt.xlabel('Number of clusters')
    plt.ylabel('Distortion')
    plt.show()
    
    kn = KneeLocator(
        range(1, max_clusters),
        distortions,
        curve='convex',
        direction='decreasing',
        interp_method='interp1d',
    )
    return kn.knee

def plot_countries(df, title, country_column_name, column_name, n_clusters, legend=False):
    world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
    merged_inner = pd.merge(left=world, right=df, left_on='name',right_on=country_column_name, how='inner')
    cmap = cm.get_cmap('Spectral', n_clusters)
    ax = world.plot(figsize=(20,5), linewidth=0.25, edgecolor='white', color='lightgrey')
    ax.set_title(title)
    
    merged_inner.plot(column=column_name, ax=ax, cmap=cmap, legend=legend)
    plt.show()

In [ ]:
def draw_scatter(df, col_1, col_2, cluster_column, num_clusters, title):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    ax.set_title(title)
    ax.set_xlabel(col_1)
    ax.set_ylabel(col_2)
    labels = list(range(0,num_clusters))
    colors = plt.cm.Spectral(np.linspace(0, 1, num_clusters))
    axs = []
    for i in labels:
        axs.append(ax.scatter(df[df[cluster_column]==i][col_1], df[df[cluster_column]==i][col_2], cmap=colors[i]))
    
    ax.legend(axs, labels, loc='center', bbox_to_anchor=(0.92, 0.84), ncol=1)
    plt.show()


In [ ]:
def create_3d_scatter(df, col_1, col_2, col_3, cluster_column, num_clusters, title):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.set_title(title)
    ax.set_xlabel(col_1)
    ax.set_ylabel(col_2)
    ax.set_zlabel(col_3, rotation=90)
    labels = list(range(0,num_clusters))
    colors = plt.cm.Spectral(np.linspace(0, 1, num_clusters))
    axs = []
    for i in labels:
        d = df[df[cluster_column]==i]
        axs.append(ax.scatter(d[col_1], d[col_2], d[col_3], cmap=colors[i]))
    
    ax.legend(axs, labels, bbox_to_anchor=(0.2, 0.5), ncol=1)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_zticklabels([])
    plt.show()

Now, we can create a dataframe from the principal components scores and use that for the clustering analysis. 

In [ ]:
pca_df = pd.DataFrame(data = pca.fit_transform(x), columns = components_names)
pca_df.head()

Use the elbow test in order to define the optimal number of clusters.

In [ ]:
n_clusters = elbow_test(pca_df, 10, 20, 300)
print(n_clusters)

In [ ]:
km = KMeans(n_clusters=n_clusters, random_state=0)
y = km.fit_predict(pca_df)
idx = np.argsort(km.cluster_centers_.sum(axis=1))
lut = np.zeros_like(idx)
lut[idx] = np.arange(n_clusters)
pca_df['cluster'] = lut[km.labels_]
df['cluster'] = lut[km.labels_]

We can see that clusters 5 has only 2 elements. 

In [ ]:
pca_df[pca_df['cluster']==5]

We can see that the PC7 value is high. It corresponds to the infection reason 'italian'. We can corroborate this by looking at the actual data:  

In [ ]:
df[df['cluster']==5]

In [ ]:
create_3d_scatter(pca_df, 'PC1', 'PC2', 'PC3', 'cluster', n_clusters, '')

From the graph we can see that some of the clusters are distributed according the first 3 principal components.  
However, In this graph, cluster 3 and 4 are not well defined. 

| Cluster | PC1  |  PC2  | PC3  |
| :------:|:----:|:-----:|:----:|
| 0       | low  |  low  | low  |
| 1       | low  |middle | low  |
| 2       | high |middle | low  |

The meaning for the principal components was defined as follows: 
- PC1: female who is not isolated and is not from from Korea
- PC2: first months 
- PC3: state released   

So, we can conclude:    

| Cluster | Meaning                                                   |
|:-------:|:----------------------------------------------------------|
|    0    | mostly men from Korea not released in the thirth month    |
|    1    | mostly men from Korea not released in the first two months|
|    2    | mostly female from France not released                    |    

   
So, let's graph other components, to see if we can find the meaning of clusters 3 and 4. For this we can draw a scatter plot. 

In [ ]:
draw_scatter(pca_df, 'PC3', 'PC5', 'cluster', n_clusters, '')

From the grap we can see that PC5 values for Cluster 3 are high. It means that corresponds to infection reason religious. We can check that with the actual data: 

In [ ]:
df[df['cluster']==3].infection_reason.unique()

There is no clear cluster definition for cluster 4.  

## Conclusion 

In summary, these are the clusters that we found:


| Cluster | Meaning                                                   |
|:-------:|:----------------------------------------------------------|
|    0    | mostly men from Korea not released in the thirth month    |
|    1    | mostly men from Korea not released in the first two months|
|    2    | mostly female from France not released                    |
|    3    | infection reason religious                                |
|    4    | others                                                    |
|    5    | infection reason italian                                  |

